In [ ]:
-- Unidades totales y clientes distintos por día
SELECT fecha, SUM(cantidad) AS unidades_totales, COUNT(DISTINCT cliente_id) AS clientes_distintos
FROM dbo.fact_ventas
GROUP BY fecha
ORDER BY fecha;

In [ ]:
-- Ticket promedio (ingreso promedio por venta)
SELECT AVG(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) AS ticket_promedio
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id;

🟢 Ejercicio: Ingreso bruto por día y su promedio diario global.

In [ ]:
SELECT fecha, SUM(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) AS ingreso_bruto
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
GROUP BY fecha
ORDER BY fecha;

SELECT AVG(x.ingreso_bruto) AS ingreso_diario_promedio
FROM (SELECT fecha, SUM(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) AS ingreso_bruto
      FROM dbo.fact_ventas v JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
      GROUP BY fecha) x;

🟠 Ejercicio: Margen bruto total (ingreso - costo).

In [ ]:
SELECT SUM(v.cantidad * p.precio_unitario * (1 - v.descuento_pct/100)) - SUM(v.cantidad * p.costo_unitario) AS margen_total
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id;

🔴 Reto: Top 3 productos por ingreso acumulado.

In [ ]:
SELECT TOP 3 p.producto_id, p.nombre, SUM(v.cantidad * p.precio_unitario) AS ingreso
FROM dbo.fact_ventas v
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
GROUP BY p.producto_id, p.nombre
ORDER BY ingreso DESC;

Errores comunes: olvidar DISTINCT en conteos; no separar costos vs ingresos; mezclar granularidades.